In [1]:
from common_imports import *
from fact_utils import setup_counterfact, COUNTERFACT_PATH, get_covariance_path
from model_utils import *
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/amakelov/workspace/current/conda_envs/serimats/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = get_model()

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
def edit_to_patch(a, b, W, Sigma, beta:float):
    Sigma_pinv = torch.linalg.pinv(Sigma)
    W_pinv = torch.linalg.pinv(W)
    # solve the linear system W Sigma^+ W^T lambda = -\beta^2 a - 2\beta Wb
    # for lambda
    lambda_ = torch.linalg.solve(W @ Sigma_pinv @ W.T, -beta**2 * a - 2 * beta * W @ b)
    # compute w as -W^+a -(2/beta)b -(1/beta^2)Sigma^+ W^T lambda
    w = -W_pinv @ a - (2/beta) * b - (1/beta**2) * Sigma_pinv @ W.T @ lambda_
    # alpha is sqrt(beta)
    alpha = np.sqrt(beta)
    # now, v is alpha * W^+a + alpha * w
    v = alpha * W_pinv @ a + alpha * w
    # finally, the objective is alpha^2 v^T Sigma v + 2 alpha b^T Sigma v
    obj = alpha**2 * v.T @ Sigma @ v + 2 * alpha * b.T @ Sigma @ v
    return v, obj